# Genetic Algorithms

Aquí estará la implementación de los algoritmos genéticos

## Genes

In [46]:
gene_set = " abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ?!."
target = "Hello World!"
target = "Que tan veloz encontraras este texto?!"

## Generate a Guess

In [47]:
import random


def generate_parent(length):
    genes = []
    while len(genes) < length:
        sample_size = min(length - len(genes), len(gene_set))
        genes.extend(random.sample(gene_set, sample_size))
    return "".join(genes)


elements = [
    "element1",
    "element2",
    "element3",
    "element4",
    "element5",
    "element6",
    "element7",
]
# values = [0.1, 0.5, 0.8, 0.3, 0.6, 0.4, 0.7]
values = [10, 50, 80, 30, 60, 40, 70]

sampled_elements = random.choices(elements, weights=values, k=2)
print(sampled_elements)

['element3', 'element2']


## Fitness


In [48]:
def get_fitness(guess):
    return sum(1 for expected, actual in zip(target, guess) if expected == actual)

## Mutate


In [49]:
def mutate(parent):
    index = random.randrange(0, len(parent))
    child_genes = list(parent)
    new_gene, alternate = random.sample(gene_set, 2)
    child_genes[index] = alternate if new_gene == child_genes[index] else new_gene
    return ''.join(child_genes)

## Display

In [50]:
import datetime

def display(guess, start_time):
    timeDiff = datetime.datetime.now() - start_time
    fitness = get_fitness(guess)
    print("{0}\t{1}\t{2}".format(guess, fitness, str(timeDiff)))

## Main


In [51]:
random.seed()
start_time = datetime.datetime.now()
best_parent = generate_parent(len(target))
best_fitness = get_fitness(best_parent)
display(best_parent, start_time)

RiDSYZpInkWQ.c ?fJPGTrgEvNtodVzsaLhHXu	0	0:00:00.000141


In [52]:
while True:
    child = mutate(best_parent)
    child_fitness = get_fitness(child)
    if best_fitness >= child_fitness:
        continue
    display(child, start_time)
    if child_fitness >= len(best_parent):
        break
    best_fitness = child_fitness
    best_parent = child

RiDSYZpInkWQ.c nfJPGTrgEvNtodVzsaLhHXu	1	0:00:00.010657
RiDSYZpIneWQ.c nfJPGTrgEvNtodVzsaLhHXu	2	0:00:00.011801
RiDSYZpIneWQ.c nfJPGTrgEvNtodV saLhHXu	3	0:00:00.011956
RiDSYZpIneWQ.c nfJPGTrgEsNtodV saLhHXu	4	0:00:00.012365
RiDSYZpInelQ.c nfJPGTrgEsNtodV saLhHXu	5	0:00:00.012822
RiDSYZpInelQ.c nfJPGTrrEsNtodV saLhHXu	6	0:00:00.013580
RiDSYZpInelQ.c nfJPGTrrEsNtodV saxhHXu	7	0:00:00.014019
RiDSYZpInelQ.c nfJPGTrrEsNtodV saxhHX!	8	0:00:00.015310
RiDSYZp nelQ.c nfJPGTrrEsNtodV saxhHX!	9	0:00:00.015793
RiDSYZp nelQ.c nfJPGTrrEsNeodV saxhHX!	10	0:00:00.015826
RiDSYZp nelQ.c nfJPGTrrEsNeodV sexhHX!	11	0:00:00.016319
RiDSYZp nelQ.c nfJPtTrrEsNeodV sexhHX!	12	0:00:00.017134
RiDSYZp nelQ.c nfJPtTrrEsNeodV sextHX!	13	0:00:00.017729
QiDSYZp nelQ.c nfJPtTrrEsNeodV sextHX!	14	0:00:00.018071
QiD YZp nelQ.c nfJPtTrrEsNeodV sextHX!	15	0:00:00.018186
QiD tZp nelQ.c nfJPtTrrEsNeodV sextHX!	16	0:00:00.021870
QiD tZp nelQ.c nfJPtrrrEsNeodV sextHX!	17	0:00:00.022351
QiD tZp nelQ.c nfJPtrrrEsNeodV textHX!	1

## Implementación en Clases

Primero hay que implementar la clase base para representar un **Gen**. Esta clase debe tener funcionalidades para generar aleatoriamente valores validos para el Gen.
Luego hay que crear una clase **Cromosoma** que se encargará de tener una lista de genes y también tendrá funcionalidades como mezclar un cromosoma con otro de su mismo tipo (que tenga el mismo tamaño y tenga los mismos tipos de genes).


### Genes


In [53]:
from typing import Self
from abc import abstractmethod


class Gene:
    """Base class for all genes"""

    def __init__(self, identifier: str, initial_value=None) -> None:
        self.identifier: str = identifier
        """The unique identifier of the Gene"""
        self.value = initial_value
        """The value of the Gene"""

    @abstractmethod
    def mutate(self) -> None:
        """Mutate the gene in place"""
        pass

    @abstractmethod
    def clone(self, with_mutation: bool = False) -> Self:
        """Returns a clone of this gene"""
        pass

    def __str__(self) -> str:
        return str(self.value)

    def __repr__(self) -> str:
        return str(self.value)

    def __eq__(self, value: object) -> bool:
        return isinstance(value, Gene) and self.identifier == value.identifier

In [54]:
import random as rnd
import uuid


class ChoiceGene(Gene):
    def __init__(
        self,
        elements: list | str,
        identifier: str = str(uuid.uuid4()),
        initial_value=None,
    ) -> None:
        super().__init__(identifier, initial_value)
        if elements is None or len(elements) == 0:
            raise Exception(f"The elements parameter should not be empty")
        self.elements: list = elements
        self.value = (
            rnd.choice(self.elements)
            if initial_value is None or initial_value not in self.elements
            else initial_value
        )

    def mutate(self) -> None:
        v1, v2 = rnd.sample(self.elements, 2)
        self.value = v1 if self.value == v2 else v2

    def clone(self, with_mutation: bool = False) -> Self:
        return ChoiceGene(
            self.elements, self.identifier, None if with_mutation else self.value
        )

    def __str__(self) -> str:
        return str(self.value)

    def __repr__(self) -> str:
        return str(self.value)

    def __eq__(self, value: object) -> bool:
        return super().__eq__(value) and isinstance(value, ChoiceGene)

### Chromosome

In [55]:
class Chromosome:
    """Base class for all chromosomes"""

    def __init__(
        self,
        genes: list[Gene],
        maximization_problem: bool,
    ) -> None:
        if genes is None or len(genes) == 0:
            raise Exception(f"The genes parameter is None or is empty")
        self.genes: list[Gene] = genes
        """The list of all the genes"""
        self.maximization_problem: bool = maximization_problem
        """Represents if the problem is a maximization problem"""

    @abstractmethod
    def mate(self, other: Self) -> Self:
        """This method mates two chromosomes and generate a new chromosome child"""
        pass

    @abstractmethod
    def mutate(self, mutation_rate: float = 0.3) -> Self:
        """Returns a mutation of this chromosome"""
        pass

    @abstractmethod
    def calculate_fitness(self) -> float:
        """A function for calculate the fitness of this chromosome"""
        pass

    def get_solution(self):
        """This method converts the genes in a solution of the domain problem"""
        pass

    @abstractmethod
    def clone(self, with_mutation: bool) -> Self:
        """Returns a clone of this chromosome"""
        pass

    def equal(self, other_chromosome: Self) -> bool:
        """Says if two chromosomes are equals, meaning that they could mate"""
        return len(self.genes) == len(other_chromosome.genes) and all(
            gene1 == gene2 for gene1, gene2 in zip(self.genes, other_chromosome.genes)
        )

In [56]:
from typing import Callable, Any


class BasicChromosome(Chromosome):
    """This chromosome mates with the single point strategy"""

    def __init__(
        self,
        genes: list[Gene],
        generate_solution: Callable[[list[Gene]], Any],
        fitness_function: Callable[[Any], float],
        maximization_problem: bool,
    ) -> None:
        super().__init__(genes, maximization_problem)
        self.generate_solution: Callable[[list[Gene]],] = generate_solution
        """This function represents how to convert the genes to a solution in the domain problem"""
        self.fitness_function: Callable[[list], float] = fitness_function
        """The fitness function"""
        self.fitness: float = None
        """The fitness of this chromosome"""
        self.dirty: bool = True

    def get_solution(self):
        return self.generate_solution(self.genes)

    def calculate_fitness(self) -> float:
        if self.fitness is not None and not self.dirty:
            return self.fitness
        solution = self.get_solution()
        self.fitness = self.fitness_function(solution)
        self.dirty = False
        return self.fitness

    def mate(self, other: Self) -> Self:
        if not self.equal(other):
            raise Exception("The chromosomes for mating are not compatibles")
        pivot_point = rnd.randrange(0, len(self.genes))
        clone = lambda genes: [gene.clone() for gene in genes]
        first1, last1 = (
            clone(self.genes[:pivot_point]),
            clone(self.genes[pivot_point:]),
        )
        first2, last2 = (
            clone(other.genes[:pivot_point]),
            clone(other.genes[pivot_point:]),
        )
        child1, child2 = (
            BasicChromosome(
                first1 + last2,
                self.generate_solution,
                self.fitness_function,
                self.maximization_problem,
            ),
            BasicChromosome(
                first2 + last1,
                self.generate_solution,
                self.fitness_function,
                self.maximization_problem,
            ),
        )
        cmp = lambda x, y: x > y if self.maximization_problem else x < y
        best = (
            child1
            if cmp(child1.calculate_fitness(), child2.calculate_fitness())
            else child2
        )
        return best

    def clone(self, with_mutation: bool) -> Self:
        cloned_genes = [gene.clone(with_mutation) for gene in self.genes]
        return BasicChromosome(
            cloned_genes,
            self.generate_solution,
            self.fitness_function,
            self.maximization_problem,
        )

    def mutate(self, mutation_rate: float = 0.3):
        for i in range(len(self.genes)):
            if rnd.random() < mutation_rate:
                self.genes[i].mutate()
        self.dirty = True
        self.calculate_fitness()

### Now the Genetic Algorithm

Este algoritmo funciona de la siguiente forma

In [57]:
class GeneticAlgorithm:
    """Base class for all implementations of Genetic Algorithms"""

    def __init__(self, sample_chromosome: Chromosome) -> None:
        self.sample_chromosome: Chromosome = sample_chromosome
        """The chromosome that represents a solution in the genetic algorithm"""
        self.maximization_problem: bool = self.sample_chromosome.maximization_problem
        """Tells if the problem to solve is a maximization problem"""

    @abstractmethod
    def selection(
        self,
        population: list[Chromosome],
    ) -> list[tuple[Chromosome, Chromosome]]:
        """This method returns a list of tuples where each tuple represents the parents to mate.
        Remember each chromosome has the fitness function inside"""
        pass

    def best_chromosome(
        self,
        chromosome1: Chromosome,
        chromosome2: Chromosome,
    ) -> Chromosome:
        """Returns the best chromosome"""
        cmp = lambda x, y: x > y if self.maximization_problem else x < y
        if cmp(chromosome1.calculate_fitness(), chromosome2.calculate_fitness()):
            return chromosome1
        return chromosome2

    def get_best_solution(self, population: list[Chromosome]) -> Chromosome:
        """Returns the best chromosome in a population"""
        if population is None or len(population) == 0:
            raise Exception(f"The population passed is None or Empty")
        best = population[0]
        # cmp = lambda x, y: x > y if self.maximization_problem else x < y
        # TODO: Make the comparison more effective by storing the fitness once and not calculating it every time
        for i in range(1, len(population)):
            actual = population[i]
            best = self.best_chromosome(actual, best)
        return best

    def mate_population(self, population: list[Chromosome]) -> list[Chromosome]:
        """This method generate a new population of individuals from a population.
        This method calls internally the selection and the cross_over method"""
        new_population: list[Chromosome] = []
        parents: list[tuple[Chromosome, Chromosome]] = self.selection(population)
        for parent1, parent2 in parents:
            child: Chromosome = parent1.mate(parent2)
            new_population.append(child)
        return new_population

    def solve(
        self,
        population_size: int,
        max_iterations: int,
        stop_criteria: Callable[
            [float], bool
        ],  # Stop criteria takes as parameter the fitness
        mutation_rate: float = 0.3,
    ) -> Chromosome:
        """This returns the population whose fitness was the best"""
        population: list[Chromosome] = [
            self.sample_chromosome.clone(True) for _ in range(population_size)
        ]
        best_solution = self.sample_chromosome.clone(True)
        for _ in range(max_iterations):
            # all_fitness = [c for c in ]
            for individual in population:
                individual.calculate_fitness()
            # Updating the best solution so far
            best_solution = self.best_chromosome(
                best_solution, self.get_best_solution(population)
            )
            # Generate new population
            new_population = self.mate_population(population)
            # Apply mutation
            for offspring in new_population:
                offspring.mutate(mutation_rate)
            # Check for stopping criteria
            if stop_criteria(best_solution.calculate_fitness()):
                return best_solution
            population = new_population
        return best_solution

In [58]:
class BasicGeneticAlgorithm(GeneticAlgorithm):
    """This class implements the genetic algorithm by using single point crossover"""

    def __init__(self, sample_chromosome: Chromosome) -> None:
        super().__init__(sample_chromosome)

    def selection(
        self, population: list[Chromosome]
    ) -> list[tuple[Chromosome, Chromosome]]:
        progenitor_list: list[tuple[Chromosome, Chromosome]] = []
        weights = [c.calculate_fitness() for c in population]
        for _ in range(len(population)):
            # idx, idy = rnd.sample(range(len(population)), 2)
            p1, p2 = rnd.choices(population, weights, k=2)
            progenitor_list.append((p1, p2))
        return progenitor_list

### Ahora chequeemos como se soluciona el problema inicial con esta nueva implementación

In [59]:
gene_set = " abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!."
target = "Hello World!"
target = "Que tan veloz encontraras este texto?!"


def get_word_fitness(guess: str):
    return sum(1 for expected, actual in zip(target, guess) if expected == actual)

In [60]:
word_gene = ChoiceGene(gene_set)


def generate_word_solution(word_gene: list[ChoiceGene]) -> str:
    solution = [gene.value for gene in word_gene]
    return "".join(solution)


def generate_word_chromosome(
    word_gene: ChoiceGene, target_length: int
) -> BasicChromosome:
    genes = [word_gene.clone(True) for _ in range(target_length)]
    sample_chromosome = BasicChromosome(
        genes, generate_word_solution, get_word_fitness, maximization_problem=True
    )
    return sample_chromosome


sample_chromosome = generate_word_chromosome(
    word_gene=word_gene, target_length=len(target)
)
algorithm = BasicGeneticAlgorithm(sample_chromosome=sample_chromosome)
result = algorithm.solve(10, 1000, lambda value: value >= len(target))
solution = result.get_solution()
print(solution)

MjuTGBOAOqPIPttXHbiRYeNmzCLvZz BecwVtc
